In [1]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np

# TIME_FRAMES = 50
BUFFER_SIZE = 100
# AMOUNT_OF_FRAMES = TIME_FRAMES - BUFFER_SIZE + 1
# AMOUNT_OF_VIDEOS = int(input("amount of videos: "))

WIDTH = 1280
HEIGHT = 720

frame_sq = []
frame_sq_gray = []
frame_sq_edges = []

# dataset_no = input("starting number: ")
video_count = 0
                    



# rgb = np.ndarray((TIME_FRAMES - (TIME_FRAMES % BUFFER_SIZE), 3))



In [2]:
def calculate_n_of_hsv(frame_sq,n):
    sum_n = 0
    var_n = 0
    for frame in frame_sq:
        var_n += np.var(frame[:,:,n])# / (np.mean(frame[:,:,n]) * 1
        sum_n += np.sum(frame[:,:,n])

    avg_n = sum_n/ (len(frame_sq) * frame_sq[0].shape[0] * frame_sq[0].shape[1])
    return np.array([avg_n ,var_n])
def calculate_comp(frame_sq):
    sum_comp = 0
    for i in range(len(frame_sq)):
        if(i != 0):
            sum_comp += compare_img(frame_sq[i], frame_sq[i-1])
#         if(i == 1): print(compare_img(frame_sq[i], frame_sq[i-1]))
    sum_comp /= (len(frame_sq) - 1)
    return sum_comp
def calculate_opticFlow(frame_sq):
    prvs = cv2.resize(frame_sq[0],(0,0),fx=0.5,fy=0.5)
    mag_sum = 0
    ang_sum = 0
    var_mag = 0
    var_ang = 0
    for i in range(1,len(frame_sq)):
        frame2 = frame_sq[i]
        next = cv2.resize(frame2, (0,0), fx=0.5, fy=0.5) 
        flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
        mag = cv2.normalize(mag,None,0,1,cv2.NORM_MINMAX)
        ang = np.absolute(cv2.normalize(ang,None,-1,1,cv2.NORM_MINMAX))
        mag_sum = np.sum(mag)
        ang_sum = np.sum(ang)
        var_mag += np.var(mag)
        var_ang += np.var(ang)
    mag_sum /= (BUFFER_SIZE * WIDTH * HEIGHT)
    ang_sum /= (BUFFER_SIZE * WIDTH * HEIGHT)
    var_mag /= BUFFER_SIZE
    var_ang /= BUFFER_SIZE
    return np.array([mag_sum, ang_sum, var_mag, var_ang])

In [3]:
def compare_img(img1,img2):
    img1 = normalize_gray(img1)
    img2 = normalize_gray(img2)
    diff = img1 - img2
    m_norm = np.sum(abs(diff))
#     return m_norm
    return 0
def normalize_frame(frame):
    return frame/255

def normalize_gray(frame):
    rng = np.max(frame) - np.min(frame)
    amin = np.min(frame)
    return (frame-amin)/rng #range from [0,1]
def createDataset(data):
    dataset = data[0].copy()
    data.remove(data[0])
    for datum in data:
        dataset = np.append(dataset, datum, axis = 1)
    return dataset
def createTSV(dataset,filename):
    dataset = (dataset.copy()).astype(str)
#     datasetName = "dataset" + str(dataset_no) + ".tsv"
    datasetName = filename + ".tsv"
    file = open(datasetName, "w")
#     file.write("h\ts\tv\tframe_edges_comp\tframe_comp\n")
    for i in range(dataset.shape[0]):
        file.write("\t".join(dataset[i,:].tolist()))
        file.write("\n")
#         if(i + 1 != dataset.shape[0]):
#             file.write("\n")
    file.close()
    return
def createTSVLabel(dataset,filename):
    dataset = (dataset.copy()).astype(str)
#     datasetName = "label" + str(dataset_no) + ".tsv"
    datasetName = filename
    file = open(datasetName, "w")
    file.write("havg\thv\tsavg\tsv\tvavg\tvv\tmagavg\tmagv\tangavg\tangv\n")
    for i in range(dataset.shape[0]):
        file.write("\t".join(dataset[i,:].tolist()))
        file.write("\n")
#         if(i + 1 != dataset.shape[0]):
#             file.write("\n")
    file.close()
    return


<h1>Normalization</h1>

In [4]:
def normalize(dataset):
    dataset = np.copy(dataset)
    datasetNan = np.isnan(dataset).any(axis=1)
    nans = []
    for i in range(datasetNan.shape[0]):
        if(datasetNan[i] == True):
            nans.append(i)
    dataset = dataset[~np.isnan(dataset).any(axis=1)]
    max_values = np.max(dataset,axis=0)
    min_values = np.min(dataset,axis=0)
    ranges = max_values - min_values
    dataset /= ranges
    return (dataset,nans)
    

In [5]:
# def createCSV(dataset):
#     dataset = (dataset.copy()).astype(str)
#     dataset_name = "dataset" + str(dataset_no) + ".csv"
#     csv = open(dataset_name, "w")
#     columnTitleRow = "havg,hv,savg,sv,vavg,vv,magavg,magv,angavg,angv\n"
#     csv.write(columnTitleRow)
#     for i in range(dataset.shape[0]):
#         csv.write(",".join(dataset[i,:].tolist()))
#         csv.write("\n")
#     csv.close()
def createCSV(dataset,filename):
    dataset = (dataset.copy()).astype(str)
#     dataset_name = "dataset" + str(dataset_no) + ".csv"
    dataset_name = filename + ".csv"
    csv = open(dataset_name, "w")
    columnTitleRow = "havg,hv,savg,sv,vavg,vv,magavg,magv,angavg,angv\n"
    csv.write(columnTitleRow)
    for i in range(dataset.shape[0]):
        csv.write(",".join(dataset[i,:].tolist()))
        csv.write("\n")
    csv.close()

In [ ]:
cap = cv2.VideoCapture(0)

frame_counter = 0
buffer_counter = 0
# filename = 'eye0.avi'
# out = cv2.VideoWriter(filename, get_video_type(filename), 25, get_dims(cap, res))

while(video_count < AMOUNT_OF_VIDEOS):
    print("video " + str(video_count) + "...")
    for i in range (BUFFER_SIZE):
        ret, frame = cap.read()
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        frame_sq.append(hsv)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_sq_gray.append(gray)
        edges = cv2.Canny(gray,100,200)
        frame_sq_edges.append(edges)
        cv2.imshow("frame",frame)
        #
        videos[video_count, i, :, :, :] = frame
        #

    h[video_count,:] = calculate_n_of_hsv(frame_sq,0)
    s[video_count,:] = calculate_n_of_hsv(frame_sq,1)
    v[video_count,:] = calculate_n_of_hsv(frame_sq,2)
    frame_edges_comp[video_count,0] = calculate_comp(frame_sq_edges)
    frame_comp[video_count,0] = calculate_comp(frame_sq_gray)
    frame_sq = []
    frame_sq_edges = []
    frame_sq_gray = []
    video_count += 1
    tmp = input('enter anything to continue')
#     (h[buffer_counter,:], h_histo) = calculate_n_of_hsv(frame_sq, 0)
#     (s[buffer_counter,:], s_histo) = calculate_n_of_hsv(frame_sq, 1)
#     (v[buffer_counter,:], v_histo) = calculate_n_of_hsv(frame_sq, 2)

dataset = createDataset([h, s, v, frame_edges_comp, frame_comp])
createTSV(dataset)
cap.release()
cv2.destroyAllWindows()

<h1>STORE VIDEOS</h1>

In [45]:
import os
import cv2
import sys

directory_name = input("directory name: ")
os.makedirs(directory_name)

BUFFER_SIZE = 100
# AMOUNT_OF_VIDEOS = 70
RECORD_AMOUNT = 20
video_count = 50

for i in range(RECORD_AMOUNT + 1):
#     print("recording video " + str(video_count) + "...")
    sys.stdout.write("\r" + "recording video " + str(video_count) + "...")
    sys.stdout.flush()
    cap = cv2.VideoCapture(0)
    WIDTH = int(cap.get(3))
    HEIGHT = int(cap.get(4))
    video_name = directory_name + "/" + str(video_count) + ".avi"
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(directory_name + "/" + str(video_count) + '.avi',fourcc, 20.0, (640,480))
    frame_counter = 0
    while(True):
        ret, frame = cap.read()
        if (ret == True):
            out.write(frame)
#             cv2.imshow('frame',frame)
            frame_counter += 1
        if (frame_counter == BUFFER_SIZE): break
    video_count += 1
#     tmp = input("press enter to continue")
    cap.release()
    cv2.destroyAllWindows()
    tmp = input("press Enter")
video_count = 0

    
    

FileExistsError: [Errno 17] File exists: 'doit3'

<h1>ANALYZE VIDEOS + PREPROCESSING</h1>

In [46]:
import os
import sys

directory = input('enter folder name: ')
vids = os.listdir(directory)
tmp = np.arange(0,len(vids) - 1)
videos = [name + '.avi' for name in (tmp.astype(str)).tolist()]
videos = [directory + "/" + name for name in videos]
video_count = 0
AMOUNT_OF_VIDEOS = len(videos) - 1

h = np.ndarray((AMOUNT_OF_VIDEOS, 2))
s = np.ndarray((AMOUNT_OF_VIDEOS, 2))
v= np.ndarray((AMOUNT_OF_VIDEOS, 2))
opticFlow = np.ndarray((AMOUNT_OF_VIDEOS,4))

print("then")
while(video_count < len(videos)-1):
#     print("video " + str(video_count) + "...")
    sys.stdout.write("\r" + "video " + str(video_count) + "...")
    sys.stdout.flush()
    cap = cv2.VideoCapture(videos[video_count])
    for i in range (BUFFER_SIZE):
        ret, frame = cap.read()
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)  
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
        
        frame_sq.append(normalize_frame(hsv))
        frame_sq_gray.append(normalize_frame(gray))
        
    h[video_count,:] = calculate_n_of_hsv(frame_sq,0)
    s[video_count,:] = calculate_n_of_hsv(frame_sq,1)
    v[video_count,:] = calculate_n_of_hsv(frame_sq,2)
    cap.release()
    cv2.destroyAllWindows()
    opticFlow[video_count,:] = calculate_opticFlow(frame_sq_gray)
    frame_sq = []
    frame_sq_gray = []
    video_count += 1
    #     (h[buffer_counter,:], h_histo) = calculate_n_of_hsv(frame_sq, 0)
#     (s[buffer_counter,:], s_histo) = calculate_n_of_hsv(frame_sq, 1)
#     (v[buffer_counter,:], v_histo) = calculate_n_of_hsv(frame_sq, 2)

datasetUnnormalized = createDataset([h,s,v,opticFlow])
# datasetNormalized = normalize(datasetUnnormalized)
# createTSV(datasetUnnormalized)
# createCSV(datasetUnnormalized)
# dataset_no = "normalizedtest"
# createTSV(datasetNormalized)
# createCSV(datasetNormalized)

cv2.destroyAllWindows()

then
video 68...

In [48]:
# print (datasetUnnormalized)
print(videos)

['doit3/0.avi', 'doit3/1.avi', 'doit3/2.avi', 'doit3/3.avi', 'doit3/4.avi', 'doit3/5.avi', 'doit3/6.avi', 'doit3/7.avi', 'doit3/8.avi', 'doit3/9.avi', 'doit3/10.avi', 'doit3/11.avi', 'doit3/12.avi', 'doit3/13.avi', 'doit3/14.avi', 'doit3/15.avi', 'doit3/16.avi', 'doit3/17.avi', 'doit3/18.avi', 'doit3/19.avi', 'doit3/20.avi', 'doit3/21.avi', 'doit3/22.avi', 'doit3/23.avi', 'doit3/24.avi', 'doit3/25.avi', 'doit3/26.avi', 'doit3/27.avi', 'doit3/28.avi', 'doit3/29.avi', 'doit3/30.avi', 'doit3/31.avi', 'doit3/32.avi', 'doit3/33.avi', 'doit3/34.avi', 'doit3/35.avi', 'doit3/36.avi', 'doit3/37.avi', 'doit3/38.avi', 'doit3/39.avi', 'doit3/40.avi', 'doit3/41.avi', 'doit3/42.avi', 'doit3/43.avi', 'doit3/44.avi', 'doit3/45.avi', 'doit3/46.avi', 'doit3/47.avi', 'doit3/48.avi', 'doit3/49.avi', 'doit3/50.avi', 'doit3/51.avi', 'doit3/52.avi', 'doit3/53.avi', 'doit3/54.avi', 'doit3/55.avi', 'doit3/56.avi', 'doit3/57.avi', 'doit3/58.avi', 'doit3/59.avi', 'doit3/60.avi', 'doit3/61.avi', 'doit3/62.avi', '

In [49]:
(normalized,nans) = normalize(datasetUnnormalized)

In [51]:
# print(normalized)
# print(nans)
# print(normalize(datasetUnnormalized))
createCSV(datasetUnnormalized,"unnorm8")
createCSV(normalized,"norm8")
createTSV(normalized,"norm8")

In [52]:
print(nans)

[]


<h1>DATASET APPEND</h1>

In [20]:
from numpy import genfromtxt
def appendDatasets():
    dataset = []
    dataset_counter = 0
    pathName = input("Path name: ")
    while(True):
        fileName = input("File name: ")
        if(fileName == ""):
            break
        next_dataset = (genfromtxt(pathName + fileName, delimiter=','))[1:,:]
        if(dataset_counter == 0):
            dataset = next_dataset
            dataset_counter += 1
        else:
            dataset = np.append(dataset, next_dataset, axis = 0)
    return dataset
def appendLabels():
    labels = []
    pathName = input("Path name: ")
    while(True):
        fileName = input("File name: ")
        if(fileName == ""):
            break
        with open(pathName + fileName) as f:
            lines = f.readlines()
        for line in lines:
            labels.append(line[:-1])
    return labels


In [11]:
dts = appendDatasets()
createCSV(dts,"Frankenstein")

In [ ]:
labels = appendLabels()
print(labels)